<br>
<font size='6'><b>Advanced Arduino</b></font><br><br>

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 60% style="border-style: hidden; border-collapse: collapse;">

        </td>
        <td width = 20%>
        by Seungchul Lee<br>iSystems Design Lab<br>http://isystems.unist.ac.kr/<br>UNIST
        </td>
    </tr>
</table>

Table of Contents
<div id="toc"></div>

# 1. Visual Output (revisited)



## 1.1. Dual 7 Segment

We have learned about 7 segments that can only represent 1 digit. However, there are many applications where we need to represent more than one digit. Let's make an embedded system that can represent two digits using a dual 7 segment (two 7 segments).

<img src = "./image_files/2_7seg.jpg" width = 400>

The first thing you’re probably wondering is: “__how can we control a total of 16 different segments (LED) if we only have ten pins on the Arduino?__”. 

To get some hints, see the below exprement.

- space vs. time

In [1]:
%%html
<iframe src="https://www.youtube.com/embed/1lA4UUCGUu0" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

In multiplexed LED applications the LED segments of all the digits are tied together and the common pins of each digit are turned ON separately by the microcontroller. 

When each digit is displayed only for several milliseconds, the eye cannot tell that the digits are not ON all the time. This way we can multiplex any number of 7-segment displays together. 

For example, to display the number 53, we have to send 5 to segments $a$ to $g$ and enable its common pin. After a few milliseconds, number 3 is sent to segments $a$ to $g$ and the common point of the second digit is enabled. When this process is __repeated continuously and fast enough__, it appears to the user that both displays are ON continuously.

 1) Send the segment bit pattern to segments $a$ to $g$<br>
 2) Enable digit 1<br>
 3) Wait for a few milliseconds<br>
 4) Disable digit 1<br>
 5) Send the segment bit pattern to segments $a$ to $g$<br>
 6) Enable digit 2<br>
 7) Wait for a few milliseconds<br>
 8) Disable digit 2<br>
 9) Repeat

<br>
<font size='4'><b>Lab 1: Displaying a Two-Digit Number</b></font>

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 25% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/2_7seg.jpg" width = 250>
        </td>
        <td width = 30% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/two_7segs.jpg" width = 250>
        </td>
        <td width = 35%>
<img src = "./image_files/Dual7-segment.jpg" width = 400>
        </td>
    </tr>
</table>

<img src = "./image_files/Advanced_Dual_seg.png" width = 500>

```c
const int gnd1 = 9;
const int gnd2 = 10;

byte digits[10][7] = {   
  {0, 0, 0, 0, 0, 0, 1}, // 0
  {1, 0, 0, 1, 1, 1, 1}, // 1
  {0, 0, 1, 0, 0, 1, 0}, // 2
  {0, 0, 0, 0, 1, 1, 0}, // 3
  {1, 0, 0, 1, 1, 0, 0}, // 4
  {0, 1, 0, 0, 1, 0, 0}, // 5
  {0, 1, 0, 0, 0, 0, 0}, // 6
  {0, 0, 0, 1, 1, 1, 1}, // 7
  {0, 0, 0, 0, 0, 0, 0}, // 8
  {0, 0, 0, 1, 1, 0, 0}  // 9
};

void displayDigit(int num) {         
  int pin = 2;
  for (int i = 0; i < 7; i++) {
    digitalWrite(pin + i, digits[num][i]);
  }
}

void dualBlinker (int one, int two) { //function for displaying numbers on each segment
  digitalWrite(gnd1, LOW);
  digitalWrite(gnd2, HIGH);
  displayDigit(one);
  delay(1.5);

  digitalWrite(gnd1, HIGH);
  digitalWrite(gnd2, LOW);
  displayDigit(two);
  delay(1.5);
}

void setup() {
  digitalWrite(gnd1, HIGH);
  digitalWrite(gnd2, HIGH);

  for (int i = 2; i < 11; i++) {
    pinMode(i, OUTPUT);
  }
}

void loop() {
  dualBlinker(2, 4);
}


```

<font size='4'><b>(Optional) Lab 2: Use LEDDisplay.h library</b></font>



```c
#include <LEDDisplay.h>

LEDDisplay *led;

void setup() {
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop() {
  led -> displayNumber(2, 1);
  led -> displayNumber(4, 0);
}

```

To see this in a slightly more dynamic example, we can adapt the example to make the LED display act as a counter (obviously only up to the number 99). 

```c
#include <LEDDisplay.h>

LEDDisplay *led;
int millisecondsPerCount;
int counter;
unsigned long lastUpdate;

void setup()  {
  millisecondsPerCount = 1000;
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop()  {
  unsigned long now = millis();
  if (now - lastUpdate > millisecondsPerCount) {
    lastUpdate = now;
    counter++;
    if (counter == 100) {
      counter = 0;
    }
  }

  int number = counter;
  for (int i = 0; i < 2; i++) {
    led -> displayNumber(number % 10, i);
    delay(2);
    number = number / 10;
  }
}
```

## 1.2. POV Display

- Persistence-Of-Vision (POV) display

- It makes use of the fact that our eyes are acting like a low pass filter.


In [2]:
%%html
<iframe src="https://www.youtube.com/embed/Md78sOI1-r8" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

 When a person sees an object, its image remains in the retina of the eye for a time interval of 1/16th of a second. This phenomenon is known as persistence of vision of the eye. This phenomenon is used in the POV Display to form images. We turn the LEDs on and off in such a way that the different images overlap each other forming letters. For example:
 
<img src = "./image_files/POV-Display-min.jpg" width = 500>

In [3]:
%%html
<iframe src="https://www.youtube.com/embed/57carjNxI9A" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

# 2. Serial Communication

## 2.1. What's happening during serial communication?

In [4]:
%%html
<iframe src="https://player.vimeo.com/video/97519477" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 2.2. Multiple values in seiral communication

In [5]:
%%html
<iframe src="https://player.vimeo.com/video/97527889" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

<font size='4'><b>Serial.write()</b></font>

- Writes binary data to the serial port. This data is sent as a byte or series of bytes; 
- to send the characters representing the digits of a number use the `print()` function instead.

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  Serial.write(45); // send a byte with the value 45
  int bytesSent = Serial.write("hello"); //send the string “hello” and return the length of the string.
  Serial.println();
}
```

- Serial.write(val)
    - val: a value to send as a single byte


- Serial.write(str)
    - str: a string to send as a series of bytes
    

- Serial.write(buf, len)
    - buf: an array to send as a series of bytes
    - len: the length of the buffer 

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  Serial.write(45); // send a byte with the value 45
  int bytesSent = Serial.write("hello", 6); //send the string “hello” and return the length of the string.
  Serial.println('-');
}
```

<font size='4'><b>Lab 3: Serial Communication with string</b></font>

- method 1

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  String msg = "";
  if (Serial.available() > 0) {
    while (Serial.available() > 0) {
      msg += char(Serial.read());
      delay(5);
    }
    Serial.println(msg);
  }
}
```

- method 2

> `Serial.readString()` reads characters from the serial buffer into a string.

```c
void setup() {
  Serial.begin(9600);
  Serial.setTimeout(50);
}

void loop() {
  String msg = "";
  if (Serial.available() > 0) {
    msg = Serial.readString();
    Serial.println(msg);
  }
}
```

<font size='4'><b>serialEvent()</b></font>

- Called when data is available. Use `Serial.read()` to capture this data.

>```c
void serialEvent() {
    //statements
}
```


```c
String inputString = "";         // a string to hold incoming data
boolean stringComplete = false;  // whether the string is complete

void setup() {  
  Serial.begin(9600);  
  inputString.reserve(200);      // reserve 200 bytes for the inputString
}

void loop() {  
  if (stringComplete) {
    Serial.println(inputString);    
    inputString = "";
    stringComplete = false;
  }
}

void serialEvent() {
  while (Serial.available()) {    
    char inChar = (char)Serial.read();    
    inputString += inChar;    
    if (inChar == '\n') {
      stringComplete = true;
    }
  }
}
```

<font size='4'><b>Lab 4: Serial Communication with integer</b></font>

- Try 1

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  int num = 0;
  if (Serial.available() > 0) {
    num = Serial.read();
    Serial.println(num);
  }
}
```

<img src="./image_files/serial_read_byte.png" width = 350>

- Try 2

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  int num = 0;
  if (Serial.available() > 0) {
    num = Serial.read() - '0';
    Serial.println(num);
  }
}
```

$\quad \;$But there are some problems...


<img src="./image_files/serial_read_problem.png" width = 350>

- Try 3

```c
long val = 0;

void setup() {
  Serial.begin(9600);
}

void loop() {
  int digit = 0;

  if (Serial.available() > 0) {
    while (Serial.available() > 0 ) {
      digit = Serial.read() - '0';
      val = val * 10 + digit;      
    }
    Serial.println(val);
    val = 0;
  }
}
```

$\quad \;$A nice try, but

<img src="./image_files/serial_trial_03.png" width = 350>

- Try 4

```c
long val = 0;

void setup() {
  Serial.begin(9600);
}

void loop() {
  int digit = 0;

  if (Serial.available() > 0) {
    while (Serial.available() > 0 ) {
      digit = Serial.read() - '0';
      val = val * 10 + digit;
      delay(5);
    }
    Serial.println(val);
    val = 0;
  }
}
```

<img src="./image_files/serial_correct.png" width = 350>

- Simple way

> `parseInt()` returns the first valid (long) integer number from the serial buffer

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  int num = 0;
  if (Serial.available() > 0) {
    num = Serial.parseInt();
    Serial.println(num);
  }
}
```

$\quad \;$But you will experiance some time delay in response

```c
void setup() {
  Serial.begin(9600);
  Serial.setTimeout(50);
}

void loop() {
  int num = 0;
  if (Serial.available() > 0) {
    num = Serial.parseInt();
    Serial.println(num);
  }
}
```

# 3. Interrupt 

You want to perform some action when a digital pin changes value and you don’t want to constantly check the pin state.

## 3.1. External Interrupt

__Interrupts vs. Polling__

- Polling:
    - Continuously poll IOs for change of value
    - Polling is like picking up your phone every few seconds to see if you have a call...
    - Cons:
        - Most polls are unneeded – value did not change
        - High CPU usage
        - Reaction time depends on #IOs

- Interrupt
    - Normal execution is interrupted when event occurs
    - Interrupts are like waiting for the phone to ring
    - Time spent in interrupt handlers should be kept as short as possible
    - Pro:
        - Processor resources are only used when necessary
    - Cons:
        - Program execution is interrupted in a non-deterministic manner


<img src = "./image_files/interrupt_flow_02.png" style="border:1px solid black", width =500>

__Interrupt Service Routine (ISR)__

- Event handler for interrupt
- Special, user-defined function for handling the interrupt

Watch the below introductory lecture on interrupt.

In [4]:
%%html
<iframe src="https://www.youtube.com/embed/M73dNDkBeT8" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

<img src = "./image_files/advanced_interrupt.png" style="border:1px solid black", width =500>


- LOW to trigger the interrupt whenever the pin is low,
- CHANGE to trigger the interrupt whenever the pin changes value
- RISING to trigger when the pin goes from low to high,
- FALLING for when the pin goes from high to low. 
    

<font size='4'><b>Lab 5</b></font>

- Try 1

```c
const int LED = 9; 
const int BUTTON = 2;

void switchPressed() {
  if (digitalRead(BUTTON) == HIGH)
    digitalWrite(LED, HIGH);
  else
    digitalWrite(LED, LOW);
}

void setup () {
  pinMode(LED, OUTPUT);
  digitalWrite(BUTTON, HIGH);
  attachInterrupt(0, switchPressed, CHANGE);
}

void loop () {
  // loop doing nothing
}
```

- Try 2

```c
int pin = 9;           // External LED
int INT0pin = 2;        // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(INT0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(INT0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

## 3.2. Timer interrupt

In [7]:
%%html
<iframe src="https://www.youtube.com/embed/m5_pFID-f-M?start=0&end=932" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

The Arduino UNO has three timers

- Timer0: An 8 bit timer used by Arduino functions `delay()`, `millis()`, and `micros()`.
- Timer1: A 16 bit timer used by the `Servo()` library
- Timer2: An 8 bit timer used by the `Tone()` library

The Arduino has 3 Timers and 6 PWM output pins. The relation between timers and PWM outputs is:

- Pins 5 and 6: controlled by timer0
- Pins 9 and 10: controlled by timer1
- Pins 11 and 3: controlled by timer2

Note: 
- You cannot use PWM on Pin 5, 6 when Arduino functions `delay()`, `millis()`, and `micros()` are used
- You cannot use PWM on Pin 9, 10 when `Servo()` library uses Timer1
- You cannot use PWM on Pin 11, 3 when `Tone()` library uses Timer2

<font size='4'><b>Lab 6</b></font>

You might need to install TimerOne.h library.

- Try 1

```c
#include "TimerOne.h"

void setup() {
  pinMode(13, OUTPUT);
  Timer1.initialize(500000);         // initialize timer1, and set a 500000 microseconds (or 1/2 second period)
  Timer1.attachInterrupt(timerIsr);  // attaches timerIsr() as a timer service routine
}

void timerIsr() {
  // Toggle LED
  digitalWrite(13, digitalRead(13) ^ 1);
}

void loop() {
}
```

- Try 2

```c
#include "TimerOne.h"

int num = 0;

void setup() {
  Serial.begin(9600);
  pinMode(13, OUTPUT);
  Timer1.initialize(1000000);
  Timer1.attachInterrupt(timerIsr);
}

void timerIsr() {
  digitalWrite(13, digitalRead(13) ^ 1);
  num += 1;
  Serial.println(num);
}

void loop() {
}
```

<font size='4'><b>Lab 7: Two Digit Counter from 0 to 99</b></font>

<img src = "./image_files/Advanced_Dual_seg.png" width = 500>

```c
#include "TimerOne.h"

int cnt = 0;

void setup() {  
  Timer1.initialize(1000000);
  Timer1.attachInterrupt(timerIsr);
}

void timerIsr() {  
  cnt += 1;  
}

void loop() {
  // add your program here...
}
```

<font size='4'><b>Timer interrupt embeded functions</b></font>

- `millis()`
- `micro()`

<br>
<font size='4'><b>millis()</b></font>

Returns the number of milliseconds since the Arduino board began running the current program. This number will overflow (go back to zero), after approximately 50 days. 

```c
unsigned long time;

void setup() {
  Serial.begin(9600);
}
void loop() {
  time = millis();
  Serial.print("Time: ");  
  //prints time since program started
  Serial.println(time);
  // wait a second so as not to send massive amounts of data
  delay(1000);
}
```

<font size='4'><b>micros()</b></font>

Returns the number of microseconds since the Arduino board began running the current program. This number will overflow (go back to zero), after approximately 70 minutes.


```c
unsigned long time;

void setup() {
  Serial.begin(9600);
}
void loop() {
  time = micros();
  Serial.print("Time: ");
  //prints time since program started
  Serial.println(time);
  // wait a second so as not to send massive amounts of data
  delay(1000);
}
```

__Note__

- 1,000 microseconds in a millisecond
- 1,000,000 microseconds in a second.

<font size='4'><b>Lab 8</b></font>

- Try 1

```c
const int ledPin =  13;
int ledState = LOW;
const long interval = 1000;              // interval at which to blink (milliseconds)

void setup() {
  pinMode(ledPin, OUTPUT);
}

void loop() {

  delay(interval);

  if (ledState == LOW)
    ledState = HIGH;
  else
    ledState = LOW;
    
  digitalWrite(ledPin, ledState);
}
```

- Try 2: Blink without `delay()`

```c
const int ledPin =  13;
int ledState = LOW;

// Generally, you should use "unsigned long" for variables that hold time
unsigned long previousMillis = 0;        // will store last time LED was updated
const long interval = 1000;              // interval at which to blink (milliseconds)

void setup() {
  pinMode(ledPin, OUTPUT);
}

void loop() {
  unsigned long currentMillis = millis();

  if (currentMillis - previousMillis >= interval) {

    previousMillis = currentMillis;

    if (ledState == LOW)
      ledState = HIGH;
    else
      ledState = LOW;

    digitalWrite(ledPin, ledState);
  }
}
```

__Finite State Machine (optional)__

- a device that can be in one of a set number of stable conditions depending on its _previous condition_ and on the _present values of its inputs_.

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 40% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/state_machine.png" width = 300>             
        </td>
        <td width = 40%>
<img src = "./image_files/FSM.png" width = 300>
        </td>
    </tr>
</table>



<font size='4'><b>Lab 9: LCD Timer</b></font>

<img src = "./image_files/lcd_new_scheme.png" width = 500>

<img src = "./image_files/LCD_Base_bb_Schem.png" width = 500>

Try 1
```c
#include <LiquidCrystal.h>

unsigned long previousMillis = 0;        // will store last time LED was updated
const long interval = 1000;              // interval at which to blink (milliseconds)
int count = 0;                           // counter 

LiquidCrystal lcd(12, 11, 5, 4, 3, 2);

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  // Print a message to the LCD.
  lcd.print("hello, world!");
}

void loop() {
  unsigned long currentMillis = millis(); // Setup counter

  if (currentMillis - previousMillis >= interval) { 
    count++;
    lcd.setCursor(0, 1);
    lcd.print(count);
    previousMillis = currentMillis;
  }
}
```
Try 2
```c
#include <LiquidCrystal.h>

LiquidCrystal lcd(12, 11, 5, 4, 3, 2);

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  // Print a message to the LCD.
  lcd.print("hello, world!");
}

void loop() {
  // set the cursor to column 0, line 1
  // (note: line 1 is the second row, since counting begins with 0):
  lcd.setCursor(0, 1);
  // print the number of seconds since reset:
  lcd.print(millis() / 1000);
}
```

# 4. Debouncing

## 4.1. Blinking LED using Button (Revisited)

<img src="./image_files/2_2 Lab - Blinking LED using Button.png" width = 500>

```c
const int buttonPin = 2;    // the number of the pushbutton pin
const int ledPin = 9;      // the number of the LED pin
bool ledState = 0;


void buttonUpdate() {

  ledState = !ledState;
}


void setup() {
  // put your setup code here, to run once:
  pinMode(buttonPin, INPUT);
  pinMode(ledPin, OUTPUT);
  attachInterrupt(0, buttonUpdate, RISING);

  digitalWrite(ledPin, ledState);
}

void loop() {

  digitalWrite(ledPin, ledState);
}
```

- Pushbuttons often generate spurious open/close transitions when pressed, due to mechanical and physical issues: 

- these transitions may be read as multiple presses in a very short time fooling the program. 

- checking twice in a short period of time to make sure the pushbutton is definitely pressed.

<img src = "./image_files/debounceswitchb.jpg" style="border:1px solid black", width =500>


## 4.2. Interrupt-based debouncing

```c
int pin = 9;           // LED on board
int interrupt0pin = 2;  // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(interrupt0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(interrupt0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

## 4.3. Software Debounce

- https://www.arduino.cc/en/Tutorial/Debounce

```c
const int buttonPin = 2;    // the number of the pushbutton pin
const int ledPin = 9;      // the number of the LED pin

// Variables will change:
int ledState = HIGH;         // the current state of the output pin
int buttonState;             // the current reading from the input pin
int lastButtonState = LOW;   // the previous reading from the input pin
long lastDebounceTime = 0;  // the last time the output pin was toggled
long debounceDelay = 50;    // the debounce time; increase if the output flickers

void setup() {
  pinMode(buttonPin, INPUT);
  pinMode(ledPin, OUTPUT);

  // set initial LED state
  digitalWrite(ledPin, ledState);
}

void loop() {
  // read the state of the switch into a local variable:
  int reading = digitalRead(buttonPin);

  // check to see if you just pressed the button
  // (i.e. the input went from LOW to HIGH),  and you've waited
  // long enough since the last press to ignore any noise:

  // If the switch changed, due to noise or pressing:
  if (reading != lastButtonState) {
    // reset the debouncing timer
    lastDebounceTime = millis();
  }

  if ((millis() - lastDebounceTime) > debounceDelay) {
    // whatever the reading is at, it's been there for longer
    // than the debounce delay, so take it as the actual current state:

    // if the button state has changed:
    if (reading != buttonState) {
      buttonState = reading;

      // only toggle the LED if the new button state is HIGH
      if (buttonState == HIGH) {
        ledState = !ledState;
      }
    }
  }

  // set the LED:
  digitalWrite(ledPin, ledState);

  // save the reading.  Next time through the loop,
  // it'll be the lastButtonState:
  lastButtonState = reading;
}
```

In [8]:
%%html
<iframe src="https://www.youtube.com/embed/5XWgr4vJkQ4" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

## 4.4. Hardware debouncing (optional)

- RC circuit

# 5. Synchronous Serial



<font size = '4'><b>Problem of serial ports</b></font>

<img src = "./image_files/asynchronous_tr.png" style="border:1px solid black", width =400>

Serial ports are __asynchronous__, meaning that no clock data is transmitted between the devices. Thus, the devices must agree on a certain data rate (9600, for example) prior to the transmission.


<img src = "./image_files/synchronous_serial.png" width =400>

Unlike the asynchronous model where no time data is transmitted, the clock data being transmitted can be noticed in the image above.

In [9]:
%%html
<iframe src="https://player.vimeo.com/video/111224388" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 5.1. SPI (Serial Peripheral Interface)

Reference: https://learn.sparkfun.com/tutorials/serial-peripheral-interface-spi?_ga=1.137392624.1133730960.1469244666

<img src = "./image_files/spi_slave.png" width =400>

In SPI, only one side generates the clock signal. The side that generates the clock is called the _master_, and the other side is called the _slave_. Only one master can be designated for SPI, but there can always be multiple slaves.


In [10]:
%%html
<iframe src="https://player.vimeo.com/video/111225512" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 5.2 I2C (Inter-Integrated Circuit)

<img src = "./image_files/I2C.png" width = 500>

<br>
Unlike SPI, I2C only requires two wires. 
- SDA (serial data) are bidirectional
- SCL (serial clock) are bidirectional

Typical I2C Transaction

<img src = "./image_files/I2C_transaction_2.png" width = 500>

- Each slave has a unique 7-bit address
- Direction bit 0 indicates write, 1 indicates read


__IMU (Inertial Measurement Unit) sensor example code__

- MPU-6050 
    - 6 axis (gyro + accelerometer)
    - MEMS motion tracking device
    - Use I2C to interface with the Arduino

<img src = "./image_files/MPU6050.png" width = 300>

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 25% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/imu_scheme.jpg" width = 400>
        </td>
        <td width = 30% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/ime_bread.jpg" width = 400>
        </td>
    </tr>
</table>

- The `wire` library is used to access I2C
- Do not need to understand the following code

```c
#include <Wire.h>

const int MPU = 0x68;   //Default address of I2C for MPU 6050
int16_t AcX, AcY, AcZ, Tmp, GyX, GyY, GyZ;

void setup() {
  Wire.begin();                   // Wire library initialization
  Wire.beginTransmission(MPU);    // Begin transmission to MPU
  Wire.write(0x6B);               // PWR_MGMT_1 register
  Wire.write(0);                  // MPU-6050 to start mode
  Wire.endTransmission(true);
  Serial.begin(9600);
}

void loop() {
  Wire.beginTransmission(MPU);      // Start transfer
  Wire.write(0x3B);                 // register 0x3B (ACCEL_XOUT_H), records data in queue
  Wire.endTransmission(false);      // Maintain connection
  Wire.requestFrom(MPU, 14, true);  // Request data to MPU
  //Reads byte by byte
  AcX = Wire.read() << 8 | Wire.read(); // 0x3B (ACCEL_XOUT_H) & 0x3C (ACCEL_XOUT_L)
  AcY = Wire.read() << 8 | Wire.read(); // 0x3D (ACCEL_YOUT_H) & 0x3E (ACCEL_YOUT_L)
  AcZ = Wire.read() << 8 | Wire.read(); // 0x3F (ACCEL_ZOUT_H) & 0x40 (ACCEL_ZOUT_L)
  Tmp = Wire.read() << 8 | Wire.read(); // 0x41 (TEMP_OUT_H) & 0x42 (TEMP_OUT_L)
  GyX = Wire.read() << 8 | Wire.read(); // 0x43 (GYRO_XOUT_H) & 0x44 (GYRO_XOUT_L)
  GyY = Wire.read() << 8 | Wire.read(); // 0x45 (GYRO_YOUT_H) & 0x46 (GYRO_YOUT_L)
  GyZ = Wire.read() << 8 | Wire.read(); // 0x47 (GYRO_ZOUT_H) & 0x48 (GYRO_ZOUT_L)

  //Prints values on Serial
  Serial.print("AcX = ");    Serial.print(AcX);
  Serial.print(" | AcY = "); Serial.print(AcY);
  Serial.print(" | AcZ = "); Serial.print(AcZ);
  Serial.print(" | Tmp = "); Serial.print(Tmp / 340.00 + 36.53);
  Serial.print(" | GyX = "); Serial.print(GyX);
  Serial.print(" | GyY = "); Serial.print(GyY);
  Serial.print(" | GyZ = "); Serial.println(GyZ);
  delay(333);
}
```

In [1]:
%%html
<iframe src="https://player.vimeo.com/video/111239036" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>